In [ ]:
from langchain_community.llms import Ollama
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader




USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
path="https://naruto.fandom.com/wiki/Itachi_Uchiha"

In [11]:

web_loader = WebBaseLoader(web_path=path)
web_data = web_loader.load()
print(web_data)

[Document(metadata={'source': 'https://naruto.fandom.com/wiki/Itachi_Uchiha', 'title': 'Itachi Uchiha | Narutopedia | Fandom', 'description': "Itachi Uchiha (うちはイタチ, Uchiha Itachi) was a shinobi of Konohagakure's Uchiha clan who served as an Anbu Captain. He later became an international criminal after murdering his entire clan, sparing only his younger brother, Sasuke. He afterwards joined the international criminal organisation known...", 'language': 'en'}, page_content='\n\n\n\nItachi Uchiha | Narutopedia | Fandom\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\tSign In\t\n\n\n\n\t\tRegister\t\n\n\n\n\n\n\n \n\n\n\n\n\n\tNarutopedia\n\n\n\n\n\n Explore\n\n \n\n\n\n\n Main Page\n\n\n\n\n Discuss\n\n\n\n\nAll Pages\n\n\n\n\nCommunity\n\n\n\n\nInteractive Maps\n\n\n\n\n\n\n\n\nSeries Info\n\n \n\n\n\n\nManga\n \n\n\n\n\nOriginal Manga\n\n\n\n\nBoruto Manga\n\n\n\n\nSasuke Retsuden\n\n\

In [25]:
#Chunking

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
documents = text_splitter.split_documents(web_data)
print(len(documents))


232


In [26]:
documents

[Document(metadata={'source': 'https://naruto.fandom.com/wiki/Itachi_Uchiha', 'title': 'Itachi Uchiha | Narutopedia | Fandom', 'description': "Itachi Uchiha (うちはイタチ, Uchiha Itachi) was a shinobi of Konohagakure's Uchiha clan who served as an Anbu Captain. He later became an international criminal after murdering his entire clan, sparing only his younger brother, Sasuke. He afterwards joined the international criminal organisation known...", 'language': 'en'}, page_content='Itachi Uchiha | Narutopedia | Fandom\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\tSign In\t\n\n\n\n\t\tRegister\t\n\n\n\n\n\n\n \n\n\n\n\n\n\tNarutopedia\n\n\n\n\n\n Explore\n\n \n\n\n\n\n Main Page\n\n\n\n\n Discuss\n\n\n\n\nAll Pages\n\n\n\n\nCommunity\n\n\n\n\nInteractive Maps\n\n\n\n\n\n\n\n\nSeries Info\n\n \n\n\n\n\nManga\n \n\n\n\n\nOriginal Manga\n\n\n\n\nBoruto Manga\n\n\n\n\nSasuke Retsuden\n\n\n\n\nKon

In [ ]:
#Embeddings converting into vectors
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="all-minilm"
)
docs=embeddings.embed_documents(documents)

# We have documents vectors in docs and type-documents in documents

In [27]:

from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents, embeddings)
vectorstore.save_local("faiss_index")

In [31]:
query = "Itachi Uchiha?"
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)


This article is about the character. For the manga chapter, see Itachi Uchiha!!.

editItachi Uchiha  [1][2][3][4]


うちはイタチ Uchiha Itachi


In [32]:
## Ollama Llama2 model
llm=Ollama(model="gemma3:1b")
output_parser=StrOutputParser()
# chain=prompt|llm|output_parser
# qs="what is nifty50?"
# op=chain.invoke({"question":qs})


C:\Users\chris\AppData\Local\Temp\ipykernel_24048\2264591185.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model="gemma3:1b")


In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatMessagePromptTemplate



prompt=ChatPromptTemplate.from_template(
   """
    Answer the following question based on the provided context:
    <context>
    {context}
    <context>
"""
)


doc_chain=create_stuff_documents_chain(llm,prompt)



bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based on the provided context:\n    <context>\n    {context}\n    <context>\n'), additional_kwargs={})])
| Ollama(model='gemma3:1b')
| StrOutputParser() kwargs={} config={'run_name': 'stuff_documents_chain'} config_factories=[]


In [35]:
from langchain_core.documents import Document
qs="What are the powers of Itachi Uchiha?"
doc_chain.invoke({
    "input":qs,
    "context":[Document(page_content=qs)]
})



'Please provide the context! I need the text you’re referring to in order to answer your question about the powers of Itachi Uchiha. 😊\n'

In [36]:
retriver=vectorstore.as_retriever()

from langchain.chains import create_retrieval_chain
retriver_chain=create_retrieval_chain(retriver,doc_chain)

In [40]:
## getting response from the model

qs="Who is  Itachi Uchiha?"
res=retriver_chain.invoke({"input":qs})

print(res)

{'input': 'Who is  Itachi Uchiha?', 'context': [Document(id='bcb5e075-50dd-4dc8-a787-5016ca18e9c4', metadata={'source': 'https://naruto.fandom.com/wiki/Itachi_Uchiha', 'title': 'Itachi Uchiha | Narutopedia | Fandom', 'description': "Itachi Uchiha (うちはイタチ, Uchiha Itachi) was a shinobi of Konohagakure's Uchiha clan who served as an Anbu Captain. He later became an international criminal after murdering his entire clan, sparing only his younger brother, Sasuke. He afterwards joined the international criminal organisation known...", 'language': 'en'}, page_content='This article is about the character. For the manga chapter, see Itachi Uchiha!!.\n\neditItachi Uchiha \xa0[1][2][3][4]\n\n\nうちはイタチ Uchiha Itachi'), Document(id='1d6b7be8-2cb6-4def-85a7-54575c98f86c', metadata={'source': 'https://naruto.fandom.com/wiki/Itachi_Uchiha', 'title': 'Itachi Uchiha | Narutopedia | Fandom', 'description': "Itachi Uchiha (うちはイタチ, Uchiha Itachi) was a shinobi of Konohagakure's Uchiha clan who served as an 

In [41]:
print(res['answer'])

Itachi Uchiha is a shinobi of Konohagakure’s Uchiha clan who served as an Anbu Captain. He later became an international criminal after murdering his entire clan, sparing only his younger brother, Sasuke. He afterwards joined the international criminal organisation known as Akatsuki, whose activity brought him into frequent conflict with Konoha and its ninja — including Sasuke who sought to avenge their clan by killing Itachi.
